# Problem 4.4 
##  - Implementation of Decision Tree with Gini Index
##  - Implementation of Pruning (Pre and Post)

In [1]:
import numpy as np
import pandas as pd
import numbers

# 1. Gini Index

$$ Gini = - \sum_{K=1}^y p_k^2 $$

In [242]:
def gini(target):
    """
    This function computes the gini value of a dataframe. Gini value is used for 
    evaluate the purity of the dataset. There is one input:
    1. target: the response column of the dataframe
    """
    
    gini = 1
    for element in pd.value_counts(target):
        p =  element/len(target)
        gini -= p**2
    return gini

In [243]:
def gini_index(target_name, attribute_name, data):
    """
    This function computes the gini index of an attribute, which has three inputs:
    1. target_name: string, the column name of response
    2. attribute_name: string, the column name of the attribute
    3. data: dataframe
    
    The class of the attribute could be numberic or object
    """
    # gini_value = gini(data[target_name])
    df_slice = data[[target_name, attribute_name]]
    
    node_info = [] # store the best split information
    
    # gini index for categorical attributes
    if not np.issubdtype(data[attribute_name].dtype, np.number):
        gini_index = 0
        for attr in pd.value_counts(data[attribute_name]).index:
            dv = df_slice[df_slice[attribute_name] == attr] # the subset of specific attribute value
            
            gini_dv = gini(dv[target_name]) # the gini of the subset
            gini_index += len(dv)/len(data) * gini_dv # compute the sum of gini 
            node_info.append(attr)
            
    # gini index for numerical attributes
    if np.issubdtype(data[attribute_name].dtype, np.number): 
        gini_index = np.inf
        
        # get all the possible splitting values
        sorted_attr = sorted(data[attribute_name].values)
        points  = [(sorted_attr[i]+sorted_attr[i+1])/2 for i in range(len(sorted_attr)-1)]
        
        # compute the gini index for each possible splitting value and find the smallest
        for point in points:
            # compute the gini for two subsets, + and -
            dv1 = df_slice[df_slice[attribute_name] < point]
            gini_dv1 = len(dv1)/len(data) * gini(dv1[target_name])
            
            dv2 = df_slice[df_slice[attribute_name] > point]
            gini_dv2 = len(dv2)/len(data) * gini(dv2[target_name])
            
            # find the smallest gini sum and its attribute name and the splitting value
            if gini_dv1+gini_dv2 < gini_index:
                gini_index = gini_dv1 + gini_dv2
                node_info = [point]

        
    return gini_index, node_info

# 3. Build Tree

In [244]:
def best_split(data, target_name):
    """
    This function returns the best split information (tree stump) of certain dataframe
    """
    
    attributes = list(data.columns) # get all the attributes
    attributes.remove(target_name) # delete the target attribute
    origin_gini_index = np.inf
    
    # compute the gini index for each attributes and find attribute with the smallest gini index
    for attr in attributes:
        gini_index_value, node_info = gini_index(target_name, attr, data) 
        if gini_index_value < origin_gini_index: 
            origin_gini_index = gini_index_value
            node_column_name = attr
            split_info = node_info
    return node_column_name, split_info

In [245]:
def majorClass(data, target_name):
    """
    Majority Function simply tells which class has more entries in given data-set
    """
    value_cnt = pd.value_counts(data[target_name])
    # np.unique(data['quality'])[np.argmax(np.unique(data['quality'],return_counts=True)[1])]
    major = list(value_cnt.index[value_cnt.values == value_cnt.max()])
    
    return major[0]

In [246]:
def partition_data(data, best_attr_name, value, isnumber = False, islarger = None):
    """
    This function return the new dataframe based on the best split information.
    It has five inputs:
    1. data: the dataframe that should be sliced
    2. best_attr_name: string, the name of the attribute which is going to be splitted on
    3. value: list, the best split values
    4. isnumber: boolean, identify if the best split value is numeric
    5. islarger: boolean, identify if the condition of splitting is 
                 'larger than the best split value'
    """
    
    if isnumber:
        if islarger:
            new_df = data[data[best_attr_name] > value]
        else: new_df = data[data[best_attr_name] < value]
    else: new_df = data[data[best_attr_name] == value]
    return new_df
    

## 3.1. Pre-Pruning

In [162]:
def pruning(traindata, validation, attribute_name,split_info, target_name, isnum=False):
    """
    This function determines if pre-pruning if necessary for a tree split.
    1. traindata : the train dataframe
    2. validation : the validation dataframe to test the generalization ability of the tree
    3. attribute_name : string, the column name of the attribute which might be splitted on
    4. split_info: list, the suggested split method
    5. target_name : string, the column name of the response
    6. isnum : boolean, whether the attribute is numeric
   
    """
    prediction_before_split = majorClass(traindata, target_name) 
    accuracy_before_split = (validation[target_name] == prediction_before_split).sum() / len(validation)
    accuracy_after_split = 0
    
    if isnum: # for numeric features
        # compute the accuracy of the sub-data with feature < threshold
        # sub train data 1 with the feature < threshold
        sub_data1 = partition_data(traindata, attribute_name, split_info[0], True)
        
        # prediction of sub train data 1
        prediction_after_split1 = majorClass(sub_data1, target_name)
        
        # sub val data 1 with the feature < threshold
        val_sub1 = validation[validation[attribute_name]<split_info[0]]
        
        # accuracy of sub val data 1
        accuracy_after_split += (val_sub1[target_name] == prediction_after_split1).sum() / len(val_sub1)
        
        # compute the accuracy of the sub-data with feature > threshold
        sub_data2 = partition_data(traindata, attribute_name, split_info[0], True, True)
        prediction_after_split2 = majorClass(sub_data2, target_name)
        val_sub2 = validation[validation[attribute_name]>split_info[0]]
        accuracy_after_split += (val_sub2[target_name] == prediction_after_split2).sum() / len(val_sub2)
        
        accuracy_after_split = accuracy_after_split / 2
        
    else: # for object features
        for value in split_info:
            sub_data = partition_data(traindata, attribute_name, value)
            prediction_after_split = majorClass(sub_data, target_name)
            val_sub = validation[validation[attribute_name]==value]
            accuracy_after_split += (val_sub[target_name]==prediction_after_split).sum()/len(val_sub)
        accuracy_after_split = accuracy_after_split/len(split_info)
        
    if accuracy_before_split < accuracy_after_split: # splitting at this node improve model
        return False # should keep this split
    else: return True # should pre-prune
    

## 3.2 Decision Tree with Pre-Pruning Option

In [247]:
def CART(data,originaldata,features, validation, target_name='quality',parent_node_class=None,preprune=False):
    """
    CART Algorithm: This function takes seven paramters:
    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset
 
    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty
    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node
    4. target_attribute_name = the name of the target attribute
    5. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.
    6. validation = the validation dataset
    7. preprune = boolean, whether prepruning is adapted
    
    """   
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_name])) <= 1:
        return np.unique(data[target_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return majorClass(originaldata, target_name)
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = majorClass(data, target_name)
        
        #Select the feature which best splits te dataset
        best_feature, best_split_info = best_split(data, target_name)
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        tree = {best_feature:{}}
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
                        
        
        #Grow a branch under the root node for each possible value of the root node feature
        if isinstance(best_split_info[0], numbers.Number): # for numeric features
            
            # if this tree is designed to be pre-pruned and after calculation, this node should be pre-pruned
            if preprune & pruning(data, validation, best_feature, best_split_info, target_name, True):
                
                # return the majority group
                return parent_node_class
            
            else:
                values = ['<'+str(best_split_info[0]), '>'+str(best_split_info[0])]
                islargers = [False, True]
                for i in range(2):
                    value = values[i]
                    sub_data = partition_data(data, best_feature, best_split_info[0], True, islargers[i])
                    sub_validation = partition_data(validation, best_feature, best_split_info[0], True, islargers[i])
                    sub_tree = CART(sub_data, originaldata, features,sub_validation, target_name, parent_node_class, preprune)

                    tree[best_feature][value] = sub_tree
        else: # for object features
            # if this tree is designed to be pre-pruned and after calculation, this node should be pre-pruned
            if preprune & pruning(data, validation, best_feature, best_split_info, target_name):
                # return the majority group
                return parent_node_class
            else:
                for value in best_split_info:
                    #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
                    sub_data = partition_data(data, best_feature, value, False)
                    sub_validation = partition_data(validation, best_feature, value, False)

                    #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
                    sub_tree = CART(sub_data, originaldata, features,sub_validation, target_name, parent_node_class, preprune)
                    #Add the sub tree, grown from the sub_dataset to the tree under the root node
                    tree[best_feature][value] = sub_tree
                    
        
        # post-prune the tree
        
        return (tree) 

## 3.4 Prediction

In [254]:
def check_float(value):
    """
    check if a string input is numeric
    """
    try:
        f = float(value)
        return True
    except: return False

In [255]:
def predict(input_df,tree,default = 'No prediction'):
    """
    Prediction of a new/unseen dataframe instance. This takes two parameters:
    1. input_df: a row of new instance with column names
    2. tree: the built decision tree
    3. default value: return the value in case of 
       new/unseen instance contains unseen attribute value.
    
    Also it is made in a recrusive manner.
    """
    for column in input_df:
        if column in list(tree.keys()):
            
            attr_value = list(input_df[column])[0]
            
            if isinstance(attr_value, numbers.Number):
                threshold = float(list(tree[column].keys())[0][1:])
                if attr_value < threshold:
                    attr_value = '<'+str(threshold)
                else: 
                    attr_value = '>'+str(threshold)
                    result = tree[column][attr_value]
            else: # do categorical classification
                try:

                    result = tree[column][attr_value] 
                except:
                    return default
  
            result = tree[column][attr_value]

            if isinstance(result,dict):
                return predict(input_df,result)
            else:
                return result

## 3.3 Post-Pruning

In [256]:
def deleting_node(tree, data, validation, target_name):
    """
    This function is checking if the start node with splitting of a tree should be merged into a node
    four input:
    1. tree: the tree, nested dictionary
    2. data: df, train data
    3. validation: df, val data
    4. target_name: string
    """
    
    origin_predict = []
    new_validation = validation.drop([target_name], axis=1)
    
    for i in range(len(validation)):
        # make prediction of validation[i] based on tree
        new_pred = predict(new_validation.iloc[[i]], tree)
        
        # store the prediction
        origin_predict.append(new_pred)
        
    # compute the number of correct prediction, original tree
    origin_correct = sum([validation[target_name].iloc[i] == origin_predict[i] for i in range(len(validation))])
    
    # the prediction of tree if the node is pruned -> the majority of the train data
    prediction_after_pruning = majorClass(data, target_name)
    
    # compute the number of correct prediction, pruned tree
    post_correct = sum([validation[target_name].iloc[i] == prediction_after_pruning for i in range(len(validation))])
    
    if origin_correct > post_correct: # if origin tree performs better
        return False # should not be pruned, the tree should be kept
    else: return True # should be pruned

In [257]:
def post_prune(tree, data, validation,target_name='quality'):
    
    best_feature = list(tree.keys())[0]
    sub_tree = tree[best_feature]
    
    for key in sub_tree.keys():
        sub_sub_tree = sub_tree[key]
        
        if isinstance(sub_sub_tree, dict): # if the sub_sub_tree is a tree, not node
            
            if check_float(key[1:]): # if the attribute is numeric
                if key[0] == '<': # if the key is: eg. <x, x: a number 
                    
                    # slice the sub train and val data
                    sub_data = partition_data(data, best_feature, float(key[1:]), True)
                    sub_validation = partition_data(validation, best_feature, float(key[1:]), True)
                elif key[0] == '>':# if the key is: eg. >x, x: a number 
                    # slice the sub train and val data
                    sub_data = partition_data(data, best_feature, float(key[1:]), True, True)
                    sub_validation = partition_data(validation, best_feature, float(key[1:]), True, True)
            
            else: # if the attribute is object
                # slice the sub train and val data
                sub_data = partition_data(data, best_feature, key, False)
                sub_validation = partition_data(validation, best_feature, key, False)
            post_prune_tree = post_prune(sub_sub_tree, sub_data, sub_validation, target_name)
            tree[best_feature][key] = post_prune_tree
        
    
    if deleting_node(tree, data, validation, target_name): # if the tree should be pruned
        return majorClass(data, target_name)
    else: 
        return tree

# 4. Load Data and Prediction

In [2]:
data = pd.read_csv('../../data/data.txt').drop(['Id'], axis=1)

In [3]:
data_tr = data.iloc[[0,1,2,5,6,9,13,14,15,16]]
data_test = data.drop(data_tr.index)
data_test_x, data_test_y = data_test.drop(['quality'],axis=1), data_test.quality

In [4]:
data_tr

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
0,dark-green,roll-up,dull,clear,hollow,hard,0.697,0.460,good
1,pitch-dark,roll-up,dead,clear,hollow,hard,0.744,0.376,good
2,pitch-dark,roll-up,dull,clear,hollow,hard,0.634,0.264,good
5,dark-green,slighly-curled,dull,clear,slightly-hollow,soft,0.403,0.237,good
6,pitch-dark,slighly-curled,dull,indistinct,slightly-hollow,soft,0.481,0.149,good
9,dark-green,stiff,crisp,clear,plain,soft,0.243,0.267,bad
13,white,slighly-curled,dead,indistinct,hollow,hard,0.657,0.198,bad
14,pitch-dark,slighly-curled,dull,clear,slightly-hollow,soft,0.360,0.370,bad
15,white,roll-up,dull,blurred,plain,hard,0.593,0.042,bad
16,dark-green,roll-up,dead,indistinct,slightly-hollow,hard,0.719,0.103,bad


In [261]:
data_test

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
3,dark-green,roll-up,dead,clear,hollow,hard,0.608,0.318,good
4,white,roll-up,dull,clear,hollow,hard,0.556,0.215,good
7,pitch-dark,slighly-curled,dull,clear,slightly-hollow,hard,0.437,0.211,good
8,pitch-dark,slighly-curled,dead,indistinct,slightly-hollow,hard,0.666,0.091,bad
10,white,stiff,crisp,blurred,plain,hard,0.245,0.057,bad
11,white,roll-up,dull,blurred,plain,soft,0.343,0.099,bad
12,dark-green,slighly-curled,dull,indistinct,hollow,hard,0.639,0.161,bad


In [262]:
data_tr_new = data_tr[['color','sound','density','quality']]
data_test_try = pd.read_csv('../data/data1.txt').drop(['Id'], axis=1)

data_tr_new

,color,sound,density,quality
0,dark-green,dull,0.697,good
1,pitch-dark,dead,0.744,good
2,pitch-dark,dull,0.634,good
5,dark-green,dull,0.403,good
6,pitch-dark,dull,0.481,good
9,dark-green,crisp,0.243,bad
13,white,dead,0.657,bad
14,pitch-dark,dull,0.360,bad
15,white,dull,0.593,bad
16,dark-green,dead,0.719,bad


In [263]:
data_test_try

,color,sound,density,quality
0,dark-green,crisp,0.697,good
1,dark-green,dull,0.697,good
2,dark-green,dull,0.300,bad
3,dark-green,dead,0.400,bad
4,white,crisp,0.400,bad
5,pitch-dark,dull,0.500,good
6,pitch-dark,dull,0.400,good
7,pitch-dark,dull,0.400,bad
8,pitch-dark,dull,0.400,bad


## 4.1 Fully Grown Tree 

In [252]:
f_tey = ['color','sound','density']

tree_1 = CART(data_tr_new,data_tr_new,f_tey, data_test_try, target_name='quality',parent_node_class=None,preprune=False)
tree_1

{'color': {'dark-green': {'sound': {'crisp': 'bad',
    'dead': 'bad',
    'dull': 'good'}},
  'pitch-dark': {'density': {'<0.4205': 'bad', '>0.4205': 'good'}},
  'white': 'bad'}}

In [267]:
data_test_try['quality'][1]

'good'

## 4.2 Pre-pruned Tree

In [264]:
tree_2 = CART(data_tr_new,data_tr_new,f_tey, data_test_try, target_name='quality',parent_node_class=None,preprune=True)
tree_2

{'color': {'dark-green': 'bad',
  'pitch-dark': {'density': {'<0.4205': 'bad', '>0.4205': 'good'}},
  'white': 'bad'}}

## 4.3 Post-pruned Tree

In [265]:
tree_1 = CART(data_tr_new,data_tr_new,f_tey, data_test_try, target_name='quality',parent_node_class=None,preprune=False)
tree_3 = post_prune(tree_1, data_tr_new, data_test_try)
tree_3

{'color': {'dark-green': 'bad',
  'pitch-dark': {'density': {'<0.4205': 'bad', '>0.4205': 'good'}},
  'white': 'bad'}}